# Script to upload contents of an mbtile to an s3 storage bucket

In [1]:
import __future__
import tinys3
import sqlite3
import progressbar
#from multiprocessing.pool import ThreadPool
#from multiprocessing import Pool, Value
import threading

In [ ]:
"""Script to upload contents of an mbtile to an s3 storage bucket"""


def main():
    """Start here"""
    print("Opening file")
    # Try to work with the mbtile file
    mbcon = sqlite3.connect(mbtile_fname)
    cur = mbcon.cursor()
    metadata = dict(mbcon.execute('select name, value from metadata;').fetchall())
    total = mbcon.execute('select count(zoom_level) from tiles;').fetchone()[0]
    print("Connecting to s3")
    # Try to connect to s3 and create our bucket
    threadLock = threading.Lock()
    bucket = tinys3.Connection(AWS_ID, AWS_SECRET_KEY, tls=True, default_bucket='bucket')
    # folders = bucket.list('', '/')
    # Set up the visuals
    widgets = [ progressbar.ETA(), progressbar.Bar()]
    pbar = progressbar.ProgressBar(widgets = widgets, maxval=total).start()
    #done = Value('i', 0)
    done = 0
    
    #for tile in mbcon.execute('select zoom_level, tile_column, tile_row, tile_data from tiles;'):
    #    name = "%s/%s/%s.png" % (tile[0], tile[1], tile[2])
    #    key = bucket.new_key(name)
    #    key.set_metadata('Content-Type', 'image/png')
    #    key.set_contents_from_string(str(tile[3]))
    #    done = done + 1
    #    pbar.update(done)
    #tiles = mbcon.execute('select zoom_level, tile_column, tile_row, tile_data from tiles;')
    #pool = ThreadPool(processes=4)
    #pool.map_async(upload, tiles, chunksize = 1)
    
    with threadLock:
        for tile in mbcon.execute('select zoom_level, tile_column, tile_row, tile_data from tiles;'):
            t = threading.Thread(target = upload, args=(tile,bucket)).start()
            done += 1
            pbar.update(done)
    #s3con = S3Connection(aws_access_key_id=AWS_ID, aws_secret_access_key=AWS_SECRET_KEY)
    #bucket = s3con.get_bucket(bucket_name)
    #with threadLock:
    #    for tile in mbcon.execute('select zoom_level, tile_column, tile_row, tile_data from tiles;'):
    #        done += 1
    #        
    #        pbar.update(done)
    # Do the work
    #for tile in mbcon.execute('select zoom_level, tile_column, tile_row, tile_data from tiles;'):
    pbar.finish()
    return(0)

In [ ]:
def upload(mytile, bucket):
    name = "%s/%s/%s.png" % (mytile[0], mytile[1], mytile[2])
    #key = bucket.new_key(name)
    bucket.upload(name, mytile[3],'my_bucket',content_type='image/png')
    #key.set_metadata('Content-Type', 'image/png')
    #key.set_contents_from_string(mytile[3])
    #key.set_acl('public-read')
    return mytile

In [ ]:

bucket_name='gif-layers'
mbtile_fname ='gif.mbtiles'
main()